In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    roc_curve,
    roc_auc_score
)
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import norm

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import norm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

---
## SECCIÓN 1: Fundamentos de Probabilidad

### 1.1 Conceptos básicos

**Probabilidad**: Medida de qué tan probable es que ocurra un evento.

$$P(A) = \frac{\text{Casos favorables}}{\text{Casos totales}}$$

**Ejemplo:** Probabilidad de sacar un as en una baraja de 52 cartas:
$$P(\text{As}) = \frac{4}{52} \approx 0.077$$

### 1.2 Probabilidad Condicional

**Probabilidad condicional**: Probabilidad de A sabiendo que B ya ocurrió.

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

**Ejemplo en medicina:** Si haces un test y da positivo, ¿cuál es la probabilidad real de estar enfermo?

In [ ]:
# Ejemplo: Test de enfermedad
P_enfermo = 0.01
P_sano = 0.99
P_test_pos_dado_enfermo = 0.99
P_test_pos_dado_sano = 0.05

P_test_pos = (P_test_pos_dado_enfermo * P_enfermo) + (P_test_pos_dado_sano * P_sano)
P_enfermo_dado_test_pos = (P_test_pos_dado_enfermo * P_enfermo) / P_test_pos

print("=" * 60)
print("EJEMPLO: TEST DE ENFERMEDAD")
print("=" * 60)
print(f"\n📋 Datos iniciales:")
print(f"  P(Enfermo) = {P_enfermo:.1%}")
print(f"  P(Test+ | Enfermo) = {P_test_pos_dado_enfermo:.1%}  (sensibilidad)")
print(f"  P(Test+ | Sano) = {P_test_pos_dado_sano:.1%}       (falso positivo)")
print(f"\n🔢 Cálculos:")
print(f"  P(Test+) = {P_test_pos:.4f}")
print(f"  P(Enfermo | Test+) = {P_enfermo_dado_test_pos:.1%}")
print(f"\n💡 Interpretación:")
print(f"  Incluso si das positivo, solo hay {P_enfermo_dado_test_pos:.1%} de probabilidad de estar enfermo")

### 1.3 El Teorema de Bayes

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

**En clasificación:** $P(\text{Clase}|\text{Datos}) = \frac{P(\text{Datos}|\text{Clase}) \cdot P(\text{Clase})}{P(\text{Datos})}$

- **P(Clase | Datos)**: Probabilidad posterior (lo que queremos)
- **P(Datos | Clase)**: Verosimilitud
- **P(Clase)**: Prior
- **P(Datos)**: Evidencia

---
## SECCIÓN 2: Naive Bayes - La Suposición Ingenua

### 2.1 ¿Por qué "Naive"?

Naive Bayes **asume que todas las características son independientes** dado la clase.

**¿Es realista?** ❌ NO, pero funciona bien en práctica porque:
- Simplifica enormemente los cálculos
- A menudo da buenos resultados
- Es especialmente bueno en análisis de textos
- Es **rápido** y necesita **pocos datos**

---
## SECCIÓN 3: Naive Bayes Gaussiano (Características Continuas)

### 3.1 Distribución Normal por clase

Asumimos que cada característica sigue una distribución normal dentro de cada clase:

$$P(x_i|\text{Clase}) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right)$$

In [ ]:
# Cargar dataset Iris
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = pd.Series(iris.target, name="species")

# Visualizar distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

feature_idx = 0
feature_name = iris.feature_names[feature_idx]

# Histograma
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    axes[0].hist(data, alpha=0.6, label=iris.target_names[class_idx], bins=15)

axes[0].set_xlabel(feature_name)
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución por Clase')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Curvas normales
x_range = np.linspace(X_iris[feature_name].min(), X_iris[feature_name].max(), 200)
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    mu = data.mean()
    sigma = data.std()
    y_dist = norm.pdf(x_range, mu, sigma)
    axes[1].plot(x_range, y_dist, lw=2, label=iris.target_names[class_idx])
    axes[1].fill_between(x_range, y_dist, alpha=0.2)

axes[1].set_xlabel(feature_name)
axes[1].set_ylabel('Densidad de Probabilidad')
axes[1].set_title('Distribuciones Normales (Asumidas)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Cada clase tiene su propia distribución normal")

### 3.2 Entrenamiento

El entrenamiento solo calcula μ y σ para cada característica en cada clase.

In [ ]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42, stratify=y_iris
)

# Entrenar modelo
nb_gaussian = GaussianNB()
nb_gaussian.fit(X_train, y_train)

# Ver parámetros
print("=" * 60)
print("PARÁMETROS APRENDIDOS")
print("=" * 60)

for class_idx, class_name in enumerate(iris.target_names):
    print(f"\n🔹 Clase: {class_name}")
    print(f"   Medias (μ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        mu = nb_gaussian.theta_[class_idx, feat_idx]
        print(f"     {feat_name}: {mu:.3f}")
    
    print(f"   Desviaciones (σ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        print(f"     {feat_name}: {sigma:.3f}")
    
    print(f"   Prior P(Clase): {nb_gaussian.class_prior_[class_idx]:.3f}")

### 3.3 Predicción manual paso a paso

Vamos a predecir la clase de un ejemplo calculando las probabilidades manualmente:

In [ ]:
# Tomar un ejemplo
sample_idx = 5
sample = X_test.iloc[sample_idx].values
true_class = y_test.iloc[sample_idx]

print("=" * 70)
print("PREDICCIÓN MANUAL CON TEOREMA DE BAYES")
print("=" * 70)
print(f"\n📊 Flor a clasificar:")
for i, feat_name in enumerate(iris.feature_names):
    print(f"   {feat_name}: {sample[i]:.2f}")
print(f"\nClase real: {iris.target_names[true_class]}")

# Función para calcular P(x|clase)
def gaussian_prob(x, mu, sigma):
    numerator = np.exp(-((x - mu)**2) / (2 * sigma**2))
    denominator = np.sqrt(2 * np.pi * sigma**2)
    return numerator / denominator

print("\n" + "=" * 70)
print("CÁLCULO DETALLADO POR CADA CLASE")
print("=" * 70)

posteriors = []
for class_idx in np.unique(y_train):
    prior = np.log(nb_gaussian.class_prior_[class_idx])
    likelihood = 0
    
    print(f"\n🔸 Clase: {iris.target_names[class_idx]}")
    print(f"   Prior: P(Clase) = {nb_gaussian.class_prior_[class_idx]:.4f}")
    print(f"   Likelihoods (características):")
    
    for feat_idx in range(len(sample)):
        mu = nb_gaussian.theta_[class_idx, feat_idx]
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        x_val = sample[feat_idx]
        
        prob = gaussian_prob(x_val, mu, sigma)
        likelihood += np.log(prob)
        print(f"     {iris.feature_names[feat_idx]}: P({x_val:.2f}|clase) = {prob:.6f}")
    
    posterior = prior + likelihood
    posteriors.append(posterior)
    print(f"   Posterior total (log): {posterior:.4f}")

print("\n" + "=" * 70)
print("RESULTADO")
print("=" * 70)
best_class_idx = np.argmax(posteriors)
print(f"\n✅ PREDICCIÓN: {iris.target_names[best_class_idx]}")
print(f"✓ Correcta: {'SÍ ✅' if best_class_idx == true_class else 'NO ❌'}")

### 3.4 Probabilidades predichas

El modelo también devuelve probabilidades (no solo la clase predicha):

In [ ]:
# Obtener probabilidades
y_pred_proba = nb_gaussian.predict_proba(X_test)

print("=" * 70)
print("PROBABILIDADES PREDICHAS (PRIMEROS 5 EJEMPLOS)")
print("=" * 70)

for i in range(5):
    print(f"\n📌 Ejemplo {i+1}:")
    print(f"   Real: {iris.target_names[y_test.iloc[i]]}")
    for class_idx, class_name in enumerate(iris.target_names):
        prob = y_pred_proba[i, class_idx]
        bar = "█" * int(prob * 20)
        print(f"   {class_name:15s}: {prob:.4f} {bar}")
    pred = nb_gaussian.predict([X_test.iloc[i].values])[0]
    print(f"   ➜ Predicción: {iris.target_names[pred]}")

### 3.5 Evaluación del modelo

Métricas estándar de clasificación:

In [ ]:
# Predicciones
y_pred = nb_gaussian.predict(X_test)

print("=" * 70)
print("EVALUACIÓN EN TEST SET")
print("=" * 70)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.tight_layout()
plt.show()

---
## SECCIÓN 4: Naive Bayes Multinomial (Para Textos)

### 4.1 Problema: Clasificación de reseñas

Queremos clasificar reseñas de películas como positivas o negativas basándonos en las palabras que contienen.

In [ ]:
# Dataset: Reseñas de películas
reviews = [
    "This movie is amazing and fantastic",
    "I loved this film absolutely wonderful",
    "Terrible movie waste of time",
    "This is a bad and boring film",
    "Great acting and excellent story",
    "Horrible and disappointing",
    "Best movie ever made",
    "Worst film I have ever seen"
]

labels = [1, 1, 0, 0, 1, 0, 1, 0]  # 1=Positivo, 0=Negativo

print("=" * 70)
print("DATASET: RESEÑAS DE PELÍCULAS")
print("=" * 70)
for i, (review, label) in enumerate(zip(reviews, labels)):
    sentiment = "POSITIVO 🟢" if label == 1 else "NEGATIVO 🔴"
    print(f"{i+1}. [{sentiment}] {review}")

### 4.2 Vectorización: Bag of Words

Convertimos cada texto en un vector de conteos de palabras:

In [ ]:
# Convertir textos a matriz de conteos
vectorizer = CountVectorizer(lowercase=True, stop_words='english')
X_text = vectorizer.fit_transform(reviews)

feature_names = vectorizer.get_feature_names_out()
print("=" * 70)
print("VOCABULARIO APRENDIDO")
print("=" * 70)
print(f"Palabras únicas: {len(feature_names)}")
print(f"Palabras: {', '.join(feature_names)}")

# Mostrar matriz de conteos
print("\n" + "=" * 70)
print("MATRIZ DE CONTEOS (BAG OF WORDS)")
print("=" * 70)
df_counts = pd.DataFrame(
    X_text.toarray(),
    columns=feature_names
)
df_counts['Sentimiento'] = ['POSITIVO' if l == 1 else 'NEGATIVO' for l in labels]
print(df_counts)

### 4.3 Entrenamiento Multinomial

Naive Bayes Multinomial trabaja directamente con conteos de palabras:

In [ ]:
# Entrenar
nb_multi = MultinomialNB()
nb_multi.fit(X_text, labels)

print("=" * 70)
print("PARÁMETROS APRENDIDOS")
print("=" * 70)

print("\n🔴 CLASE NEGATIVA (0):")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[0]):
    print(f"     {word:15s}: {np.exp(prob):.4f}")

print("\n🟢 CLASE POSITIVA (1):")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[1]):
    print(f"     {word:15s}: {np.exp(prob):.4f}")

print(f"\n📊 Probabilidades iniciales (priors):")
print(f"   P(NEGATIVO) = {np.exp(nb_multi.class_log_prior_[0]):.4f}")
print(f"   P(POSITIVO) = {np.exp(nb_multi.class_log_prior_[1]):.4f}")

### 4.4 Predicción en textos nuevos

Clasificamos textos que el modelo nunca ha visto:

In [ ]:
# Textos nuevos
new_reviews = [
    "This movie is amazing",
    "Terrible and boring film"
]

X_new = vectorizer.transform(new_reviews)
predictions = nb_multi.predict(X_new)
probabilities = nb_multi.predict_proba(X_new)

print("=" * 70)
print("PREDICCIONES EN TEXTOS NUEVOS")
print("=" * 70)

for review, pred, probs in zip(new_reviews, predictions, probabilities):
    sentiment = "POSITIVO 🟢" if pred == 1 else "NEGATIVO 🔴"
    print(f"\n📝 Texto: \"{review}\"")
    print(f"   Predicción: {sentiment}")
    print(f"   P(NEGATIVO) = {probs[0]:.4f}")
    print(f"   P(POSITIVO) = {probs[1]:.4f}")

### 4.5 Palabras más informativas

¿Qué palabras son más importantes para cada sentimiento?

In [ ]:
# Diferencia entre clases
logprob_pos = nb_multi.feature_log_prob_[1]
logprob_neg = nb_multi.feature_log_prob_[0]
diff = logprob_pos - logprob_neg

# Top 5
top_pos_idx = np.argsort(diff)[-5:]
top_neg_idx = np.argsort(diff)[:5]

print("=" * 70)
print("PALABRAS MÁS INFORMATIVAS")
print("=" * 70)

print("\n🟢 PALABRAS ASOCIADAS A POSITIVO:")
for idx in reversed(top_pos_idx):
    word = feature_names[idx]
    print(f"   {word}")

print("\n🔴 PALABRAS ASOCIADAS A NEGATIVO:")
for idx in top_neg_idx:
    word = feature_names[idx]
    print(f"   {word}")

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

words_pos = feature_names[top_pos_idx]
diffs_pos = diff[top_pos_idx]
ax1.barh(words_pos, diffs_pos, color='green', alpha=0.7)
ax1.set_xlabel('Diferencia de Log-Probabilidad')
ax1.set_title('Palabras Indicadoras de Reseña POSITIVA')
ax1.grid(True, alpha=0.3, axis='x')

words_neg = feature_names[top_neg_idx]
diffs_neg = diff[top_neg_idx]
ax2.barh(words_neg, diffs_neg, color='red', alpha=0.7)
ax2.set_xlabel('Diferencia de Log-Probabilidad')
ax2.set_title('Palabras Indicadoras de Reseña NEGATIVA')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---
## SECCIÓN 5: Caso Real - Detección de Spam

Clasificar emails como spam o legítimos usando palabras clave:

In [ ]:
# Dataset
spam_words = [
    "Click here now",
    "Limited time offer",
    "Free money today",
    "Act now immediately",
    "Call now"
]

ham_words = [
    "Hi how are you",
    "Meeting tomorrow",
    "Thanks for your help",
    "See you soon",
    "Have a great day"
]

all_texts = spam_words + ham_words
all_labels = [1]*len(spam_words) + [0]*len(ham_words)

print("=" * 70)
print("DATASET: SPAM vs EMAILS LEGÍTIMOS")
print("=" * 70)

print("\n🔴 SPAM:")
for text in spam_words:
    print(f"   {text}")

print("\n🟢 LEGÍTIMO (HAM):")
for text in ham_words:
    print(f"   {text}")

# Vectorizar
vectorizer_spam = CountVectorizer(lowercase=True, stop_words='english')
X_spam = vectorizer_spam.fit_transform(all_texts)

# Entrenar
nb_spam = MultinomialNB()
nb_spam.fit(X_spam, all_labels)

print(f"\n✅ Accuracy en training: {nb_spam.score(X_spam, all_labels):.1%}")

# Predecir en textos nuevos
new_emails = [
    "Click here for amazing offer",
    "Hi let's meet tomorrow"
]

X_new_emails = vectorizer_spam.transform(new_emails)
spam_pred = nb_spam.predict(X_new_emails)
spam_proba = nb_spam.predict_proba(X_new_emails)

print("\n" + "=" * 70)
print("PREDICCIONES")
print("=" * 70)

for email, pred, proba in zip(new_emails, spam_pred, spam_proba):
    status = "⚠️ SPAM" if pred == 1 else "✅ LEGÍTIMO"
    print(f"\n📧 {email}")
    print(f"   Clasificación: {status}")
    print(f"   Confianza: {max(proba):.1%}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import norm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

### 1.2 Probabilidad Condicional

**Probabilidad condicional**: Probabilidad de A sabiendo que B ya ocurrió.

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

**Ejemplo en medicina:**
Si haces un test y da positivo, ¿cuál es la probabilidad real de estar enfermo?

In [ ]:
# Ejemplo: Test de enfermedad
# P(Enfermo) = 0.01 (1% de población está enferma)
# P(Test+ | Enfermo) = 0.99 (99% de sensibilidad)
# P(Test+ | Sano) = 0.05 (5% de falsos positivos)

P_enfermo = 0.01
P_sano = 0.99
P_test_pos_dado_enfermo = 0.99
P_test_pos_dado_sano = 0.05

# Probabilidad total de dar test positivo
P_test_pos = (P_test_pos_dado_enfermo * P_enfermo) + (P_test_pos_dado_sano * P_sano)

# TEOREMA DE BAYES: P(Enfermo | Test+)
P_enfermo_dado_test_pos = (P_test_pos_dado_enfermo * P_enfermo) / P_test_pos

print("=" * 60)
print("EJEMPLO: TEST DE ENFERMEDAD")
print("=" * 60)
print(f"\n📋 Datos iniciales:")
print(f"  P(Enfermo) = {P_enfermo:.1%}")
print(f"  P(Test+ | Enfermo) = {P_test_pos_dado_enfermo:.1%}  (sensibilidad)")
print(f"  P(Test+ | Sano) = {P_test_pos_dado_sano:.1%}       (falso positivo)")

print(f"\n🔢 Cálculos:")
print(f"  P(Test+) = {P_test_pos:.4f}")
print(f"  P(Enfermo | Test+) = {P_enfermo_dado_test_pos:.4f} = {P_enfermo_dado_test_pos:.1%}")

print(f"\n💡 Interpretación:")
print(f"  Incluso si das positivo, solo hay {P_enfermo_dado_test_pos:.1%} de probabilidad de estar enfermo")
print(f"  ¿Por qué? Porque la enfermedad es rara (1% en población)")

### 1.3 El Teorema de Bayes

El Teorema de Bayes nos permite calcular probabilidades "hacia atrás":

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

**En contexto de clasificación:**
$$P(\text{Clase}|\text{Datos}) = \frac{P(\text{Datos}|\text{Clase}) \cdot P(\text{Clase})}{P(\text{Datos})}$$

**Significado:**
- **P(Clase | Datos)**: Probabilidad posterior (lo que queremos saber)
- **P(Datos | Clase)**: Verosimilitud (likelihood)
- **P(Clase)**: Probabilidad previa (prior)
- **P(Datos)**: Evidencia (normalizador)

---
## SECCIÓN 2: Naive Bayes - La Suposición Ingenua

### 2.1 ¿Por qué "Naive"?

Naive Bayes **asume que todas las características son independientes** dado la clase.

**¿Es realista?** ❌ NO, pero funciona bien en práctica porque:
- Simplifica enormemente los cálculos
- A menudo da buenos resultados
- Es especialmente bueno en análisis de textos
- Es **rápido** y necesita **pocos datos**

---
## SECCIÓN 3: Naive Bayes Gaussiano (Características Continuas)

### 3.1 Distribución Normal por clase

Asumimos que cada característica sigue una distribución normal dentro de cada clase:

$$P(x_i|\text{Clase}) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right)$$

In [ ]:
# Cargar dataset Iris
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = pd.Series(iris.target, name="species")

# Visualizar distribución de una característica por clase
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

feature_idx = 0
feature_name = iris.feature_names[feature_idx]

# Histograma para cada clase
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    axes[0].hist(data, alpha=0.6, label=iris.target_names[class_idx], bins=15)

axes[0].set_xlabel(feature_name)
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Sepal Length por Clase')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Curvas normales teóricas
x_range = np.linspace(X_iris[feature_name].min(), X_iris[feature_name].max(), 200)
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    mu = data.mean()
    sigma = data.std()
    y_dist = norm.pdf(x_range, mu, sigma)
    axes[1].plot(x_range, y_dist, lw=2, label=iris.target_names[class_idx])
    axes[1].fill_between(x_range, y_dist, alpha=0.2)

axes[1].set_xlabel(feature_name)
axes[1].set_ylabel('Densidad de Probabilidad')
axes[1].set_title('Distribuciones Normales (Asumidas por Naive Bayes)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Cada clase tiene su propia distribución normal")

### 3.2 Entrenamiento

El entrenamiento solo calcula la media (μ) y desviación estándar (σ) para cada característica en cada clase.

In [ ]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42, stratify=y_iris
)

# Entrenar modelo
nb_gaussian = GaussianNB()
nb_gaussian.fit(X_train, y_train)

# Ver parámetros aprendidos
print("=" * 60)
print("PARÁMETROS APRENDIDOS")
print("=" * 60)

for class_idx, class_name in enumerate(iris.target_names):
    print(f"\n🔹 Clase: {class_name}")
    print(f"   Medias (μ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        mu = nb_gaussian.theta_[class_idx, feat_idx]
        print(f"     {feat_name}: {mu:.3f}")
    
    print(f"   Desviaciones (σ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        print(f"     {feat_name}: {sigma:.3f}")
    
    prior = nb_gaussian.class_prior_[class_idx]
    print(f"   Prior P(Clase): {prior:.3f}")

### 3.3 Predicción manual paso a paso

Vamos a predecir la clase de un ejemplo calculando las probabilidades manualmente:

In [ ]:
# Tomar un ejemplo
sample_idx = 5
sample = X_test.iloc[sample_idx].values
true_class = y_test.iloc[sample_idx]

print("=" * 70)
print("PREDICCIÓN MANUAL CON TEOREMA DE BAYES")
print("=" * 70)
print(f"\n📊 Flor a clasificar:")
for i, feat_name in enumerate(iris.feature_names):
    print(f"   {feat_name}: {sample[i]:.2f}")
print(f"\nClase real: {iris.target_names[true_class]}")

# Función para calcular P(x | clase)
def gaussian_prob(x, mu, sigma):
    """Calcula P(x|clase) usando distribución normal"""
    numerator = np.exp(-((x - mu)**2) / (2 * sigma**2))
    denominator = np.sqrt(2 * np.pi * sigma**2)
    return numerator / denominator

print("\n" + "=" * 70)
print("CÁLCULO DETALLADO POR CADA CLASE")
print("=" * 70)

posteriors = []
for class_idx in np.unique(y_train):
    prior = np.log(nb_gaussian.class_prior_[class_idx])
    likelihood = 0
    
    print(f"\n🔸 Clase: {iris.target_names[class_idx]}")
    print(f"   Prior: P(Clase) = {nb_gaussian.class_prior_[class_idx]:.4f}")
    print(f"   Likelihoods (características):")
    
    for feat_idx in range(len(sample)):
        mu = nb_gaussian.theta_[class_idx, feat_idx]
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        x_val = sample[feat_idx]
        
        prob = gaussian_prob(x_val, mu, sigma)
        likelihood += np.log(prob)
        
        print(f"     {iris.feature_names[feat_idx]}: P({x_val:.2f}|clase) = {prob:.6f}")
    
    posterior = prior + likelihood
    posteriors.append(posterior)
    print(f"   Posterior total (log): {posterior:.4f}")

print("\n" + "=" * 70)
print("RESULTADO")
print("=" * 70)
best_class_idx = np.argmax(posteriors)
print(f"\n✅ PREDICCIÓN: {iris.target_names[best_class_idx]}")
print(f"   (Mayor log-posterior: {posteriors[best_class_idx]:.4f})")
print(f"\n✓ Predicción correcta: {'SÍ ✅' if best_class_idx == true_class else 'NO ❌'}")

### 3.4 Probabilidades predichas

El modelo también devuelve probabilidades (no solo la clase predicha):

In [ ]:
# Obtener probabilidades para nuevos ejemplos
y_pred_proba = nb_gaussian.predict_proba(X_test)

print("=" * 70)
print("PROBABILIDADES PREDICHAS (PRIMEROS 5 EJEMPLOS)")
print("=" * 70)

for i in range(5):
    print(f"\n📌 Ejemplo {i+1}:")
    print(f"   Real: {iris.target_names[y_test.iloc[i]]}")
    for class_idx, class_name in enumerate(iris.target_names):
        prob = y_pred_proba[i, class_idx]
        bar = "█" * int(prob * 20)
        print(f"   {class_name:15s}: {prob:.4f} {bar}")
    pred = nb_gaussian.predict([X_test.iloc[i].values])[0]
    print(f"   ➜ Predicción: {iris.target_names[pred]}")

### 3.5 Evaluación del modelo Gaussiano

Métricas estándar de clasificación:

In [ ]:
# Predicciones
y_pred = nb_gaussian.predict(X_test)

# Evaluación
print("=" * 70)
print("EVALUACIÓN EN TEST SET")
print("=" * 70)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Naive Bayes Gaussiano')
plt.tight_layout()
plt.show()

---
## SECCIÓN 4: Naive Bayes Multinomial (Para Textos)

### 4.1 Problema: Clasificación de reseñas

Queremos clasificar reseñas de películas como positivas o negativas basándonos en las palabras que contienen.

In [ ]:
# Dataset: Reseñas de películas
reviews = [
    "This movie is amazing and fantastic",
    "I loved this film absolutely wonderful",
    "Terrible movie waste of time",
    "This is a bad and boring film",
    "Great acting and excellent story",
    "Horrible and disappointing",
    "Best movie ever made",
    "Worst film I have ever seen"
]

labels = [1, 1, 0, 0, 1, 0, 1, 0]  # 1=Positivo, 0=Negativo

print("=" * 70)
print("DATASET: RESEÑAS DE PELÍCULAS")
print("=" * 70)
for i, (review, label) in enumerate(zip(reviews, labels)):
    sentiment = "POSITIVO 🟢" if label == 1 else "NEGATIVO 🔴"
    print(f"{i+1}. [{sentiment}] {review}")

### 4.2 Vectorización: Bag of Words

Convertimos cada texto en un vector de conteos de palabras:

In [ ]:
# Convertir textos a matriz de conteos
vectorizer = CountVectorizer(lowercase=True, stop_words='english')
X_text = vectorizer.fit_transform(reviews)

print("=" * 70)
print("VOCABULARIO APRENDIDO")
print("=" * 70)
feature_names = vectorizer.get_feature_names_out()
print(f"Palabras únicas: {len(feature_names)}")
print(f"Palabras: {', '.join(feature_names)}")

# Mostrar matriz de conteos
print("\n" + "=" * 70)
print("MATRIZ DE CONTEOS (BAG OF WORDS)")
print("=" * 70)
df_counts = pd.DataFrame(
    X_text.toarray(),
    columns=feature_names
)
df_counts['Sentimiento'] = ['POSITIVO' if l == 1 else 'NEGATIVO' for l in labels]
print(df_counts)

### 4.3 Entrenamiento Multinomial

Naive Bayes Multinomial trabaja directamente con conteos de palabras:

In [ ]:
# Entrenar
nb_multi = MultinomialNB()
nb_multi.fit(X_text, labels)

print("=" * 70)
print("PARÁMETROS APRENDIDOS")
print("=" * 70)

# Log-probabilidades por palabra y clase
print("\n🔴 CLASE NEGATIVA (0):")
print("   Probabilidades de cada palabra:")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[0]):
    print(f"     {word:15s}: {np.exp(prob):.4f}")

print("\n🟢 CLASE POSITIVA (1):")
print("   Probabilidades de cada palabra:")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[1]):
    print(f"     {word:15s}: {np.exp(prob):.4f}")

# Priors
print(f"\n📊 Probabilidades iniciales (priors):")
print(f"   P(NEGATIVO) = {np.exp(nb_multi.class_log_prior_[0]):.4f}")
print(f"   P(POSITIVO) = {np.exp(nb_multi.class_log_prior_[1]):.4f}")

### 4.4 Predicción en textos nuevos

Clasificamos textos que el modelo nunca ha visto:

In [ ]:
# Textos nuevos
new_reviews = [
    "This movie is amazing",
    "Terrible and boring film"
]

# Vectorizar y predecir
X_new = vectorizer.transform(new_reviews)
predictions = nb_multi.predict(X_new)
probabilities = nb_multi.predict_proba(X_new)

print("=" * 70)
print("PREDICCIONES EN TEXTOS NUEVOS")
print("=" * 70)

for review, pred, probs in zip(new_reviews, predictions, probabilities):
    sentiment = "POSITIVO 🟢" if pred == 1 else "NEGATIVO 🔴"
    print(f"\n📝 Texto: \"{review}\"")
    print(f"   Predicción: {sentiment}")
    print(f"   P(NEGATIVO) = {probs[0]:.4f}")
    print(f"   P(POSITIVO) = {probs[1]:.4f}")

### 4.5 Palabras más informativas

¿Qué palabras son más importantes para cada sentimiento?

In [ ]:
# Diferencia entre clases para cada palabra
logprob_pos = nb_multi.feature_log_prob_[1]
logprob_neg = nb_multi.feature_log_prob_[0]
diff = logprob_pos - logprob_neg

# Top 5 palabras positivas y negativas
top_pos_idx = np.argsort(diff)[-5:]
top_neg_idx = np.argsort(diff)[:5]

print("=" * 70)
print("PALABRAS MÁS INFORMATIVAS")
print("=" * 70)

print("\n🟢 PALABRAS MÁS ASOCIADAS A POSITIVO:")
for idx in reversed(top_pos_idx):
    word = feature_names[idx]
    print(f"   {word}")

print("\n🔴 PALABRAS MÁS ASOCIADAS A NEGATIVO:")
for idx in top_neg_idx:
    word = feature_names[idx]
    print(f"   {word}")

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

words_pos = feature_names[top_pos_idx]
diffs_pos = diff[top_pos_idx]
ax1.barh(words_pos, diffs_pos, color='green', alpha=0.7)
ax1.set_xlabel('Diferencia de Log-Probabilidad')
ax1.set_title('Palabras Indicadoras de Reseña POSITIVA')
ax1.grid(True, alpha=0.3, axis='x')

words_neg = feature_names[top_neg_idx]
diffs_neg = diff[top_neg_idx]
ax2.barh(words_neg, diffs_neg, color='red', alpha=0.7)
ax2.set_xlabel('Diferencia de Log-Probabilidad')
ax2.set_title('Palabras Indicadoras de Reseña NEGATIVA')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---
## SECCIÓN 5: Caso Real - Detección de Spam

Clasificar emails como spam o legítimos usando palabras clave:

In [ ]:
# Calcular la diferencia de log-probabilidades para cada palabra
feature_names = vectorizer.get_feature_names_out()
logprob_pos = nb_multi.feature_log_prob_[1]
logprob_neg = nb_multi.feature_log_prob_[0]

# Palabras más asociadas a positivo
pos_diff = logprob_pos - logprob_neg
top_pos_idx = np.argsort(pos_diff)[-5:]
top_neg_idx = np.argsort(pos_diff)[:5]

print("=" * 70)
print("PALABRAS MÁS INFORMATIVAS")
print("=" * 70)

print("\n🟢 PALABRAS MÁS ASOCIADAS A POSITIVO:")
for idx in reversed(top_pos_idx):
    word = feature_names[idx]
    diff = pos_diff[idx]
    print(f"   {word}: diferencia = {diff:.4f}")

print("\n🔴 PALABRAS MÁS ASOCIADAS A NEGATIVO:")
for idx in top_neg_idx:
    word = feature_names[idx]
    diff = pos_diff[idx]
    print(f"   {word}: diferencia = {diff:.4f}")

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Palabras positivas
words_pos = feature_names[top_pos_idx]
diffs_pos = pos_diff[top_pos_idx]
ax1.barh(words_pos, diffs_pos, color='green', alpha=0.7)
ax1.set_xlabel('Log-prob(Positivo) - Log-prob(Negativo)')
ax1.set_title('Palabras Asociadas a Reseñas Positivas')
ax1.grid(True, alpha=0.3, axis='x')

# Palabras negativas
words_neg = feature_names[top_neg_idx]
diffs_neg = pos_diff[top_neg_idx]
ax2.barh(words_neg, diffs_neg, color='red', alpha=0.7)
ax2.set_xlabel('Log-prob(Positivo) - Log-prob(Negativo)')
ax2.set_title('Palabras Asociadas a Reseñas Negativas')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---
## 5. PARTE III: Caso Real - Análisis de Spam

Usaremos un dataset real de emails spam vs legítimos:

In [ ]:
# Predicciones en test set
y_pred = nb_gaussian.predict(X_test)

# Evaluación
print("=" * 70)
print("EVALUACIÓN EN TEST SET")
print("=" * 70)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Naive Bayes Gaussiano (Iris)')
plt.tight_layout()
plt.show()

---
## 4. PARTE II: Naive Bayes Multinomial (Para Textos y Conteos)

### 4.1 Problema: Clasificación de Textos

Queremos clasificar textos según su sentimiento (positivo/negativo) o detectar spam.

**Idea:** Contar la frecuencia de palabras en cada texto.

In [ ]:
# Dataset de ejemplo: Reseñas de películas
reviews = [
    "This movie is amazing and fantastic",
    "I loved this film, absolutely wonderful",
    "Terrible movie, waste of time",
    "This is a bad and boring film",
    "Great acting and excellent story",
    "Horrible and disappointing",
    "Best movie ever made",
    "Worst film I have ever seen"
]

# Labels: 1 = Positivo, 0 = Negativo
labels = [1, 1, 0, 0, 1, 0, 1, 0]

print("=" * 70)
print("DATASET: RESEÑAS DE PELÍCULAS")
print("=" * 70)
for i, (review, label) in enumerate(zip(reviews, labels)):
    sentiment = "POSITIVO" if label == 1 else "NEGATIVO"
    print(f"{i+1}. [{sentiment}] {review}")

### 4.2 Vectorizar el texto: Bag of Words

Convertimos cada texto en un vector de conteos de palabras:

In [ ]:
# Convertir textos a matriz de conteos
vectorizer = CountVectorizer(lowercase=True, stop_words='english')
X_text = vectorizer.fit_transform(reviews)

# Ver vocabulario
print("=" * 70)
print("VOCABULARIO APRENDIDO")
print("=" * 70)
print(f"Palabras únicas: {len(vectorizer.get_feature_names_out())}")
print(f"Palabras: {', '.join(vectorizer.get_feature_names_out())}")

# Mostrar matriz de conteos
print("\n" + "=" * 70)
print("MATRIZ DE CONTEOS (BAG OF WORDS)")
print("=" * 70)
df_counts = pd.DataFrame(
    X_text.toarray(),
    columns=vectorizer.get_feature_names_out()
)
df_counts['Label'] = ['POS' if l == 1 else 'NEG' for l in labels]
print(df_counts)

### 4.3 Entrenar Naive Bayes Multinomial

Para textos, usamos Naive Bayes Multinomial que trabaja directamente con conteos de palabras:

In [ ]:
# Entrenar Naive Bayes Multinomial
nb_multi = MultinomialNB()
nb_multi.fit(X_text, labels)

print("=" * 70)
print("PARÁMETROS APRENDIDOS")
print("=" * 70)

# Log-probabilidades aprendidas
feature_names = vectorizer.get_feature_names_out()
print("\nLog-probabilidades de cada palabra por clase:")
print(f"\n🔸 Clase NEGATIVA (0):")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[0]):
    print(f"   {word}: {prob:.4f} (prob = {np.exp(prob):.4f})")

print(f"\n🔸 Clase POSITIVA (1):")
for word, prob in zip(feature_names, nb_multi.feature_log_prob_[1]):
    print(f"   {word}: {prob:.4f} (prob = {np.exp(prob):.4f})")

# Prior de cada clase
print(f"\nPrior (ocurrencia de cada clase):")
print(f"   P(NEGATIVO) = {nb_multi.class_log_prior_[0]:.4f} (prob = {np.exp(nb_multi.class_log_prior_[0]):.4f})")
print(f"   P(POSITIVO) = {nb_multi.class_log_prior_[1]:.4f} (prob = {np.exp(nb_multi.class_log_prior_[1]):.4f})")

### 4.4 Predicción en textos nuevos

Veamos cómo el modelo clasifica textos que no ha visto:

### 1.2 Probabilidad Condicional

**Probabilidad condicional**: Probabilidad de A sabiendo que B ya ocurrió.

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

**Ejemplo en medicina:**
- P(Enfermo | Test positivo) = ?
- Necesitamos saber:
  - Qué probabilidad tiene un enfermo de dar test positivo
  - Qué probabilidad tiene un sano de dar test positivo (falso positivo)

In [ ]:
# Ejemplo: Test de enfermedad
# P(Enfermo) = 0.01 (1% de la población está enferma)
# P(Test+ | Enfermo) = 0.99 (99% de sensibilidad)
# P(Test+ | Sano) = 0.05 (5% de falsos positivos)

P_enfermo = 0.01
P_sano = 0.99
P_test_pos_dado_enfermo = 0.99
P_test_pos_dado_sano = 0.05

# Probabilidad de dar test positivo
P_test_pos = (P_test_pos_dado_enfermo * P_enfermo) + (P_test_pos_dado_sano * P_sano)

# TEOREMA DE BAYES: P(Enfermo | Test+)
P_enfermo_dado_test_pos = (P_test_pos_dado_enfermo * P_enfermo) / P_test_pos

print("=" * 60)
print("EJEMPLO: TEST DE ENFERMEDAD")
print("=" * 60)
print(f"\nDatos iniciales:")
print(f"  P(Enfermo) = {P_enfermo:.1%}")
print(f"  P(Test+ | Enfermo) = {P_test_pos_dado_enfermo:.1%}  (sensibilidad)")
print(f"  P(Test+ | Sano) = {P_test_pos_dado_sano:.1%}       (falso positivo)")

print(f"\nCálculos:")
print(f"  P(Test+) = {P_test_pos:.4f}")
print(f"  P(Enfermo | Test+) = {P_enfermo_dado_test_pos:.4f} = {P_enfermo_dado_test_pos:.1%}")

print(f"\n💡 Interpretación:")
print(f"  Incluso si das positivo, solo hay {P_enfermo_dado_test_pos:.1%} de probabilidad de estar enfermo")
print(f"  ¿Por qué? Porque la enfermedad es rara (1% en población)")

### 1.3 El Teorema de Bayes (La Joya de la Corona)

El Teorema de Bayes nos permite calcular probabilidades "hacia atrás":

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

**En contexto de clasificación:**
$$P(\text{Clase}|\text{Datos}) = \frac{P(\text{Datos}|\text{Clase}) \cdot P(\text{Clase})}{P(\text{Datos})}$$

**Término a término:**
- **P(Clase | Datos)**: Lo que queremos saber (probabilidad posterior)
- **P(Datos | Clase)**: Verosimilitud (likelihood)
- **P(Clase)**: Probabilidad previa (prior)
- **P(Datos)**: Evidencia (normalizador)

---
## 2. Naive Bayes: La Suposición Ingenua

### 2.1 ¿Por qué "Naive"?

Naive Bayes **asume que todas las características son independientes** dado la clase.

$$P(\text{Datos}|\text{Clase}) = P(x_1|\text{Clase}) \cdot P(x_2|\text{Clase}) \cdot ... \cdot P(x_n|\text{Clase})$$

**¿Es realista esta suposición?** ❌ NO, muchas características están correlacionadas.

**¿Entonces por qué funciona?** ✅ Porque:
1. Simplifica enormemente los cálculos
2. A menudo da buenos resultados en la práctica
3. Es especialmente bueno en textos (palabras casi independientes)
4. Es **rápido** y necesita **poco datos**

---
## 3. PARTE I: Naive Bayes Gaussiano (Características Continuas)

### 3.1 Idea: Distribución Normal

Asumimos que cada característica sigue una **distribución normal** dentro de cada clase:

$$P(x_i|\text{Clase}) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x_i - \mu)^2}{2\sigma^2}\right)$$

Donde:
- $\mu$ = media de la característica en esa clase
- $\sigma$ = desviación estándar en esa clase

In [ ]:
# Cargar dataset Iris
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = pd.Series(iris.target, name="species")

# Visualizar distribución de una característica por clase
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

feature_idx = 0  # Sepal length
feature_name = iris.feature_names[feature_idx]

# Graficar histograma para cada clase
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    axes[0].hist(data, alpha=0.6, label=iris.target_names[class_idx], bins=15)

axes[0].set_xlabel(feature_name)
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Sepal Length por Clase')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graficar curvas normales
x_range = np.linspace(X_iris[feature_name].min(), X_iris[feature_name].max(), 200)
for class_idx in np.unique(y_iris):
    data = X_iris[y_iris == class_idx][feature_name]
    mu = data.mean()
    sigma = data.std()
    
    # Función de densidad normal
    y_dist = norm.pdf(x_range, mu, sigma)
    axes[1].plot(x_range, y_dist, lw=2, label=iris.target_names[class_idx])
    axes[1].fill_between(x_range, y_dist, alpha=0.2)

axes[1].set_xlabel(feature_name)
axes[1].set_ylabel('Densidad de Probabilidad')
axes[1].set_title('Distribución Normal Asumida por Naive Bayes')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Observa cómo cada clase tiene su propia distribución normal")

### 3.2 Entrenar Naive Bayes Gaussiano

El entrenamiento es muy simple: solo calcula $\mu$ y $\sigma$ para cada característica en cada clase.

In [ ]:
# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42, stratify=y_iris
)

# Entrenar Naive Bayes Gaussiano
nb_gaussian = GaussianNB()
nb_gaussian.fit(X_train, y_train)

# El modelo aprendió las medias y desviaciones
print("=" * 60)
print("PARÁMETROS APRENDIDOS POR NAIVE BAYES")
print("=" * 60)

for class_idx, class_name in enumerate(iris.target_names):
    print(f"\n🔹 Clase: {class_name}")
    print(f"   Medias (μ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        print(f"     {feat_name}: {nb_gaussian.theta_[class_idx, feat_idx]:.3f}")
    
    print(f"   Desviaciones estándar (σ):")
    for feat_idx, feat_name in enumerate(iris.feature_names):
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        print(f"     {feat_name}: {sigma:.3f}")
    
    print(f"   Prior P(Clase): {nb_gaussian.class_prior_[class_idx]:.3f}")

### 3.3 Predicción: Calculando probabilidades manualmente

Vamos a predecir la clase de un ejemplo step-by-step usando Bayes:

In [ ]:
# Tomar un ejemplo del test set
sample_idx = 0
sample = X_test.iloc[sample_idx].values
true_class = y_test.iloc[sample_idx]

print("=" * 70)
print("PREDICCIÓN MANUAL CON TEOREMA DE BAYES")
print("=" * 70)
print(f"\n📊 Ejemplo a clasificar:")
for i, feat_name in enumerate(iris.feature_names):
    print(f"   {feat_name}: {sample[i]:.3f}")

print(f"\nClase real: {iris.target_names[true_class]}")

# Calcular P(características | clase) para cada clase
def gaussian_prob(x, mu, sigma):
    """Calcula P(x|clase) usando distribución normal"""
    numerator = np.exp(-((x - mu)**2) / (2 * sigma**2))
    denominator = np.sqrt(2 * np.pi * sigma**2)
    return numerator / denominator

print("\n" + "=" * 70)
print("CÁLCULO DETALLADO")
print("=" * 70)

posteriors = []
for class_idx in np.unique(y_train):
    prior = np.log(nb_gaussian.class_prior_[class_idx])
    likelihood = 0
    
    print(f"\n🔸 Clase: {iris.target_names[class_idx]}")
    print(f"   Prior P(Clase) = {nb_gaussian.class_prior_[class_idx]:.4f}")
    print(f"   log(Prior) = {prior:.4f}")
    
    print(f"   Likelihoods P(características|Clase):")
    
    for feat_idx in range(len(sample)):
        mu = nb_gaussian.theta_[class_idx, feat_idx]
        sigma = np.sqrt(nb_gaussian.var_[class_idx, feat_idx])
        x_val = sample[feat_idx]
        
        prob = gaussian_prob(x_val, mu, sigma)
        likelihood += np.log(prob)
        
        print(f"      {iris.feature_names[feat_idx]}: P({x_val:.3f}|clase) = {prob:.6f}, log = {np.log(prob):.4f}")
    
    # Posterior (sin normalizar, pero suficiente para comparar)
    posterior = prior + likelihood
    posteriors.append(posterior)
    
    print(f"   log(Posterior) = {posterior:.4f}")

print("\n" + "=" * 70)
print("RESULTADO")
print("=" * 70)
best_class_idx = np.argmax(posteriors)
print(f"\n✅ Predicción: {iris.target_names[best_class_idx]}")
print(f"   (Mayor log-posterior: {posteriors[best_class_idx]:.4f})")

# Comparar con predicción del modelo
pred_model = nb_gaussian.predict([sample])[0]
print(f"\n🤖 Predicción del modelo: {iris.target_names[pred_model]}")
print(f"   Coincide: {'✅ SÍ' if pred_model == best_class_idx else '❌ NO'}")

### 3.4 Probabilidades predichas

Naive Bayes también puede darnos las probabilidades de cada clase (no solo la clase):